In [303]:
import pandas as pd
import time
import string
from functions.capitalization import titlecase
so_file='BIGBANG1.txt'
widths = [4,2,15,10,30,100,4,30,3,7,1,30,30,2,20,2,2,10,9,50,3,30,50]
update=pd.read_fwf(so_file,
                   header=None, 
                   widths=widths,
                   names=['dir_code','so_type','sonumber','mem_wstd','first_name','last_name','sam_city'
                          ,'sam_stname','sam_stsubt','sam_stnmfr','sam_subd','sam_estate','sam_bldname','sam_aptcod','sam_aprmnt'
                          ,'list_code','acc_type','old_wstd','so_date','joint_user','regulatory_id','account_no','distribution_code'],
                   converters = {'mem_wstd': str, 'sam_stnmfr':str,'account_no':str},
                   index_col=None
                  )
#update.()
update = update.fillna('')

update.sort_values(by='last_name',inplace=True)
update.reset_index(drop=True,inplace=True)
update['acc_type']=update['acc_type'].astype('category')
update['src']=so_file.split('/')[-1]
update['so_rangedate']=time.strftime("%Y/%m/%d")
rr=update

In [331]:
#rr.loc[rr.dir_code == 'MM','so_type']=400
rr_crm.name1=rr_crm.name1.str.lower()
rr_crm

,AREACODE,Phone,name1,name2,SAM_BLDNAME,SAM_STNMFR,SAM_STNAME,SAM_STSUBT,sam_estate,City,PROVINCE,class_code,class_desc,acc_type
0,NaN,7063050,,,Sm Megamall A,,Edsa,Ave,,Mandaluyong City,METRO MANILA,,,BR
1,NaN,3983090,"""k"" line logistics (phils.), inc.",,Oyster Complex Building,,Ninoy Aquino,Ave,,Paranaque City,METRO MANILA,,,BR
2,NaN,8251854,"""k"" line logistics (phils.), inc.",,,,Ninoy Aquino,Ave,,Paranaque City,METRO MANILA,,,BR
3,NaN,5562917,"""k"" line maritime training corporation",,K-Line Building,,Coralway,Drv,CENTRAL BUSINESS PARK 1,Pasay City,METRO MANILA,,,BR
4,NaN,6318181,(bank of the phil islands),,Mayflower Plaza Building,,United,St,,Mandaluyong City,METRO MANILA,,,BR
5,NaN,8967916,(department of trade and industrybsmbd),,Dti Building,361,Sen Gil Puyat,Ave,,Makati City,METRO MANILA,,,GO
6,NaN,8975554,(department of trade and industrycic),,Dti Building,361,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,METRO MANILA,,,GO
7,NaN,8908907,(department of trade and industryipr),,Dti Building,361,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,METRO MANILA,,,GO
8,NaN,8904951,(department of trade and industryoop),,Industry & Investment Building,385,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,METRO MANILA,,,GO
9,NaN,8971719,(department of trade and industryticci),,Lc Building,375,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,METRO MANILA,,,GO


In [322]:
rr_crm=pd.DataFrame()
rr_crm['AREACODE']=2
rr_crm['Phone']=rr.mem_wstd.str.slice(-7)
rr_crm['name1']=rr.last_name
rr_crm['name2']=rr.first_name
rr_crm['SAM_BLDNAME']=rr.sam_bldname
rr_crm['SAM_STNMFR']=rr.sam_stnmfr
rr_crm['SAM_STNAME']=rr.sam_stname
rr_crm['SAM_STSUBT']=rr.sam_stsubt
rr_crm['sam_estate']=rr.sam_estate
rr_crm['City']=rr.distribution_code.str.split('  ',1).str.get(1).apply(lambda x : x.strip())
rr_crm['PROVINCE']=rr.distribution_code.str.split('  ',1).str.get(0).apply(lambda x : x.strip())
rr_crm['class_code']=""
rr_crm['class_desc']=""
rr_crm['acc_type']=rr.acc_type
rr_crm.name1 = rr_crm.name1.apply(lambda x: titlecase(x) if x.isupper() else x)
rr_crm.name2 = rr_crm.name2.apply(lambda x: titlecase(x) if x.isupper() else x)
rr_crm.SAM_BLDNAME = rr_crm.SAM_BLDNAME.apply(lambda x: titlecase(x) if x.isupper() else x)
rr_crm.SAM_STNAME = rr_crm.SAM_STNAME.apply(lambda x: titlecase(x))
rr_crm.SAM_STSUBT = rr_crm.SAM_STSUBT.apply(lambda x: titlecase(x))
rr_crm.sam_estate = rr_crm.sam_estate.apply(lambda x: titlecase(x))
rr_crm.City = rr_crm.City.apply(lambda x: titlecase(x))
#rr_crm.PROVINCE = rr_crm.PROVINCE.apply(lambda x: titlecase(x))

In [323]:
import numpy as np
def or_call(crm):
    index=crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])
    skip=False
    for i in xrange(len(index)):
        if index.ix[i] == True:
            line=crm.ix[i]
            prev_line=crm.ix[i-1]
            try:
                next_line=crm.ix[i+1]
            except:
                pass
            line['SAM_BLDNAME']=''
            line['SAM_STNMFR']=''
            line['SAM_STNAME']=''
            line['SAM_STSUBT']=''
            line['sam_estate']=''
            line['PROVINCE']=''
            line['class_code']=''
            line['class_desc']=''
            #line['name1']=''
            #line['name2']=''
            if skip == False:
                prev_line['Phone']=str(prev_line.Phone)+'/'+str(line.Phone)
                line['Phone']=''

                if index.ix[i-1] == False and index.ix[i+1]==True:
                    next_line['City']='Or Call:'
                try:
                    if index.ix[i+1] == True:
                        skip =True
                    
                except:
                    pass
            else:
                skip = False
    crm['Phone'].replace('', np.nan, inplace=True)
    return  crm[crm.Phone.notnull()]

    

In [ ]:
import csv 
with open('areacodes') as csvfile:
    reader= csv.reader(csvfile)
    codes = dict((rows[0],rows[1]) for rows in reader)


In [ ]:
codes.get('bulacan')

In [ ]:
print rr_crm.ix[5]
print rr_crm.ix[6]

In [ ]:
rr_crm.duplicated('name1')

In [ ]:
def or_call(crm):
    index=crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])
    skip=False
    for i in xrange(len(index)):
        if index.iloc[i] == True:
            line=crm.ix[i]
            prev_line=crm.ix[i-1]
            try:
                next_line=crm.ix[i+1]
            except:
                pass
            line['SAM_BLDNAME']=''
            line['SAM_STNMFR']=''
            line['SAM_STNAME']=''
            line['SAM_STSUBT']=''
            line['sam_estate']=''
            line['PROVINCE']=''
            line['class_code']=''
            line['class_desc']=''
            #line['name1']=''
            #line['name2']=''
            if skip == False:
                prev_line['Phone']=str(prev_line.Phone)+'/'+str(line.Phone)
                line['Phone']=np.nan

                if index.ix[i-1] == False and index.ix[i+1]==True:
                    next_line['City']='Or Call:'
                try:
                    if index.ix[i+1] == True:
                        skip =True
                except:
                    pass
            else:
                skip = False
    #crm['Phone'].replace('', np.nan, inplace=True)
    return  crm[crm.Phone.notnull()]
or_call(rr_crm)

In [ ]:
str(open('small_words').read())

In [ ]:
open('small_words').read().rstrip('\n')


In [ ]:
import functions.interface_functions

In [ ]:
import glob
import os
newest = min(glob.iglob('database_backups/*.hdf5'), key=os.path.getctime)


In [ ]:
newest

In [ ]:
rr.distribution_code.str.split('    ',1).str.get(1)

In [ ]:
index=rr_crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])


In [ ]:
range(len(index))

In [ ]:
index[1]

In [ ]:
index=rr_crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])


In [ ]:
'dfdsf sdfsdfs        sdfsd'.split('s')

In [ ]:
rr

In [ ]:
x=8
x=x or 5
x

In [ ]:
os.getlogin()

In [ ]:
import csv
reader = csv.reader(open('config/input_format'))#,delimiter=',',quotechar='')
titles=reader.next()
widths=reader.next()

In [ ]:
titles[0]

In [ ]:
reader= csv.reader(open('config/input_format'),quoting=csv.QUOTE_NONNUMERIC)
lines=[]
for row in reader:
    print list(row)
names=lines[0]
widths=lines[1]

In [ ]:
import ConfigParser
Config = ConfigParser.ConfigParser()
Config.read("config/input_format")
Config.read('config/areacodes')
def ConfigSectionMap(section):
    dict1 = {}
    options = Config.options(section)
    for option in options:
        try:
            dict1[option] = Config.get(section, option)
            if dict1[option] == -1:
                DebugPrint("skip: %s" % option)
        except:
            print("exception on %s!" % option)
            dict1[option] = None
    return dict1

In [ ]:
Config.options('areacodes')
#ConfigSectionMap('input_format').keys()

In [ ]:
ConfigSectionMap('areacodes')
    

In [56]:
import pandas as pd
import numpy as np
crm=pd.DataFrame(np.array([['AREACODE',10],['b',1],['c',None],['n',5]]))

In [57]:
crm.sort_values(by=0)
crm.sort_values(by=0).drop_duplicates(0,keep='last')

,0,1
0,AREACODE,10
1,b,1
2,c,None
3,n,5


In [58]:
crm

,0,1
0,AREACODE,10
1,b,1
2,c,None
3,n,5


In [245]:
product=pd.read_excel('Company_Class.xlsx')
product.name1=product.name1.str.upper()
#product.Phone=5361557
rr_crm.name1=rr_crm.name1.str.upper()
rr_crm=rr_crm.drop_duplicates('Phone')

In [234]:
rr_crm.drop_duplicates('Phone').Phone=3

/Users/ncolina/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [249]:
pd.merge(rr_crm,product, on=['Phone'], how='left')

,AREACODE,Phone,name1_x,name2_x,SAM_BLDNAME,SAM_STNMFR,SAM_STNAME,SAM_STSUBT,sam_estate,City,PROVINCE,class_code_x,class_desc,acc_type,Areacode,name1_y,name2_y,Product,class_code_y
0,NaN,10,NaN,,SM MEGAMALL A,,EDSA,AVE,,MANDALUYONG CITY,ncr,,,BR,2,S C TAN EXPORT CORP,NaN,MLABUS2016_MNLYP,104376


In [240]:
rr_crm['Phone']

0        10
1        10
2        10
3        10
4        10
5        10
6        10
7        10
8        10
9        10
10       10
11       10
12       10
13       10
14       10
15       10
16       10
17       10
18       10
19       10
20       10
21       10
22       10
23       10
24       10
25       10
26       10
27       10
28       10
29       10
         ..
57770    10
57771    10
57772    10
57773    10
57774    10
57775    10
57776    10
57777    10
57778    10
57779    10
57780    10
57781    10
57782    10
57783    10
57784    10
57785    10
57786    10
57787    10
57788    10
57789    10
57790    10
57791    10
57792    10
57793    10
57794    10
57795    10
57796    10
57797    10
57798    10
57799    10
Name: Phone, dtype: int64

In [248]:
product.Phone=10
product=product.drop_duplicates('Phone')

In [263]:
Config.read("config/crm_format")

Config.options('crm_format')

ParsingError: File contains parsing errors: config/crm_format
	[line  2]: 'Areacode\n'
	[line  3]: 'Phone\n'
	[line  4]: 'name1\n'
	[line  5]: 'name2\n'
	[line  6]: 'SAM_BLDNAME\n'
	[line  7]: 'SAM_STNMFR\n'
	[line  8]: 'SAM_STNAME\n'
	[line  9]: 'SAM_STSUBT\n'
	[line 10]: 'sam_estate\n'
	[line 11]: 'City\n'
	[line 12]: 'Province\n'
	[line 13]: 'class_code\n'
	[line 14]: 'acc_type\n'

In [ ]:
crmout=['Phone', 'Areacode']
out=pd.DataFrame
for field in crmout:
    out['%s'%field]= crm['%s'%field]